In [202]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy

In [203]:
%cd C:\Users\Zak White\AppData\Local\Programs\Python\Python36\Thinkful Datasets
    
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")
#df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv')


C:\Users\Zak White\AppData\Local\Programs\Python\Python36\Thinkful Datasets


In [236]:
df.info()
# Looking at the dataframe info we can see many rows missing their PMID
# and another row missing their Journal title.
# The column titles are also messy
# The most glaring problem is there is no consistent data type for any column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [240]:
df.columns = ['PMID/PMCID', 'Publisher', 'Journal_title', 'Article_title',
       'COST(£)']
df.columns

Index(['ID', 'Publisher', 'Journal_title', 'Article_title', 'COST(£)'], dtype='object')

In [241]:
df.nunique()
#Looking at the number of uniques we can see a possible duplicate Article Title
#Possibly multiple issues with the PMID/PMCID column
# It appears there are plenty of articles that are priced similarly which is not an issue.

ID               1880
Publisher         299
Journal_title     984
Article_title    2126
COST(£)          1402
dtype: int64

In [230]:
# NOT NEEDED FOR ASSIGNMENT, BUT GOOD PRACTICE!!!
# Clean IDs
    # PMID = integers of varrying length
    # PMCIDs = PMC+integer
    # remove any row with out a "cleanable" ID
df['PMID/PMCID'].unique()

def condense(x):
    if len(x)<=8 and len(x)>6:
        return x
    else:
        return "NaN"

#Currently clean_ID splits the ID column into a table of elements
    #elements are deliniated from spaces
    #then filtered for digits/ID that are >6 in length, but no more than 8
def clean_ID(x):
    x= pd.Series(str(x).strip().split(" "))
    ##x = list(filter(lambda x: str(x).isdigit(),x))
    ##x = x.apply(lambda x: ''.join(list(filter(str.isdigit, str(x)))))
    x = x.apply(lambda x: condense(''.join(list(filter(str.isdigit, str(x))))))
    return x
ID_table = df['PMID/PMCID'].apply(clean_ID)

#
#condense this table into 1 
#len(ID_table[0][1])
#ID_table

In [231]:
import currency_converter as conv

In [232]:
#Clean price data
    # Eliminate currency unit notation
    # convert to numeric
    # If USD
        # Convert to (£)
    # Elimi
# look for "£" & "$" sign on left or right
    #if "$":
        # drop
        #convert to "£"
    #else
        # drop "£"
        
 
 def convert(x):
    x= pd.Series(str(x).strip()
    x=x.apply()
    
#print(*df['COST(£)'].unique(), sep='\n')

In [ ]:
# Dirty list
money = pd.Series([400, 111, '$20', 57, 'Lots'])

# Here's a lambda function that mirrors the is_a_digit function above.
# Read this print statement carefully and compare to the previous one.
print(money.apply(lambda x: str(x).isdigit()))